# Predspracovanie údajov

In [ ]:
# imports
import pandas as pd
import numpy as np
import hashlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer, QuantileTransformer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

## 1. Realizácia predspracovania dát

In [436]:
connections = pd.read_csv('data/connections_cleaned.csv')
connections['ts'] = pd.to_datetime(connections['ts'])

processes = pd.read_csv('data/processes_cleaned.csv')
processes['ts'] = pd.to_datetime(processes['ts'])

conn_processes = pd.merge(connections, processes, on=['ts', 'imei', 'mwra'], how='inner')
conn_processes.drop_duplicates(inplace=True)

conn_processes.head(2)

,ts,imei,mwra,c.android.gm,c.android.youtube,c.katana,c.dogalize,c.android.chrome,c.android.vending,c.raider,...,p.dogalize,p.android.defcontainer,p.google,p.android.gms,p.olauncher,p.android.vending,p.browser.provider,p.process.gapps,p.gms.persistent,p.simulator
0,2018-05-05 10:00:00,3590433799317661073,1.0,10.73779,7.60358,10.30550,12.72137,17.14262,63.02014,52.99201,...,49.63998,52.22598,86.79731,88.93302,12.84506,2.63115,90.52801,90.51873,35.23943,93.56427
1,2018-05-05 10:01:00,3590433799317661784,1.0,13.88095,10.68630,8.36266,13.90403,13.28416,54.83081,41.20946,...,50.35593,45.19193,87.84568,25.99896,90.22100,71.68478,89.94562,71.77292,64.49298,42.81274


In [437]:
devices = pd.read_csv('data/devices_cleaned.csv')
profiles = pd.read_csv('data/profiles_cleaned.csv')

### (A)

In [438]:
X = conn_processes.drop(columns=['mwra'])
y = conn_processes['mwra']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Train Dataset Shape: ', X_train.shape, y_train.shape)
print(f'Test Dataset Shape: ', X_test.shape, y_test.shape)

Train Dataset Shape:  (11835, 32) (11835,)
Test Dataset Shape:  (2959, 32) (2959,)


### (B)

In [439]:
def hash_md5(value):
    hash_object = hashlib.md5(value.encode())
    hash_int = int(hash_object.hexdigest(), 16)
    return hash_int % (2 ** 63)

In [440]:
non_numeric = X_train.select_dtypes(exclude=['number']).columns.to_list()
print('Non-numeric columns:', non_numeric)

Non-numeric columns: ['ts']


In [441]:
X_train['ts'] = X_train['ts'].astype('int64')

print('Column types in order:', X_train.dtypes.to_list())
X_train.head(2)

Column types in order: [dtype('int64'), dtype('int64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64')]


,ts,imei,c.android.gm,c.android.youtube,c.katana,c.dogalize,c.android.chrome,c.android.vending,c.raider,c.UCMobile.intl,...,p.dogalize,p.android.defcontainer,p.google,p.android.gms,p.olauncher,p.android.vending,p.browser.provider,p.process.gapps,p.gms.persistent,p.simulator
10308,1526133600000000000,3590433799317661545,15.07576,10.24261,12.55076,10.81983,16.56803,51.08433,55.40703,42.16911,...,59.00193,15.19691,24.61623,47.64864,36.44968,75.69554,0.81417,10.06329,35.59590,4.49113
10797,1526162700000000000,359043379931766635,11.47352,11.52160,14.50189,9.98012,14.12950,70.55549,66.92780,16.73113,...,60.71290,33.17497,83.04305,45.18686,27.33930,61.00963,77.18931,84.50057,46.84404,93.53882


In [442]:
non_numeric = devices.select_dtypes(exclude=['number']).columns.to_list()
print('Non-numeric columns:', non_numeric)

Non-numeric columns: ['store_name', 'code', 'continent', 'city']


In [443]:
devices['code'], code_unique = pd.factorize(devices['code'])
devices = pd.get_dummies(devices, columns=['continent'], prefix='continent')

for col in devices.columns:
    if col.startswith('continent_'):
        devices[col] = devices[col].astype(int)

devices['store_name'] = devices['store_name'].apply(hash_md5)

frequency = devices['city'].value_counts()
devices['city'] = devices['city'].map(frequency)

print('Column types in order:', devices.dtypes.to_list())
devices.head(2)

Column types in order: [dtype('float64'), dtype('float64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64')]


,latitude,longitude,store_name,code,imei,city,continent_Africa,continent_America,continent_Asia,continent_Atlantic,continent_Australia,continent_Europe,continent_Indian,continent_Pacific
0,15.58552,102.42587,1238261730008270783,0,3590433799317661925,36,0,0,1,0,0,0,0,0
1,52.59319,13.32127,3703756712017643536,1,8630330696303482139,136,0,0,0,0,0,1,0,0


In [444]:
non_numeric = profiles.select_dtypes(exclude=['number']).columns.to_list()
print('Non-numeric columns:', non_numeric)

Non-numeric columns: ['residence', 'username', 'name', 'ssn', 'registration', 'address', 'company', 'mail', 'latitude', 'longitude', 'domain']


In [445]:
profiles.drop(['residence', 'username', 'name', 'mail'], inplace=True, axis=1)

profiles['latitude'] = profiles['latitude'].replace('Unknown', 0)
profiles['longitude'] = profiles['longitude'].replace('Unknown', 0)

profiles['latitude'] = profiles['latitude'].astype(float)
profiles['longitude'] = profiles['longitude'].astype(float)

profiles['registration'] = pd.to_datetime(profiles['registration'], format='%m/%d/%Y')
profiles['registration'] = (profiles['registration']).astype(int)

def convert_ssn(value):
    cleaned = ''.join(filter(str.isdigit, value))
    return int(cleaned)

profiles['ssn'] = profiles['ssn'].apply(convert_ssn)

def encode_address(value):
    return 0 if value == 'Unknown' else 1

profiles['address'] = profiles['address'].apply(encode_address)
profiles['company'] = profiles['company'].apply(hash_md5)

profiles = pd.get_dummies(profiles, columns=['domain'], prefix='domain')

for col in profiles.columns:
    if col.startswith('domain_'):
        profiles[col] = profiles[col].astype(int)

print('Column types in order:', devices.dtypes.to_list())
profiles.head(2)

Column types in order: [dtype('float64'), dtype('float64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64'), dtype('int64')]


,imei,ssn,registration,address,company,user_id,latitude,longitude,domain_gmail.com,domain_hotmail.com,domain_yahoo.com
0,359043379931766536,380331115,1667260800000000000,1,2562683759997207315,322,29.254521,56.881839,0,1,0
1,3590433799317662154,244952123,1701129600000000000,1,8266524727329204724,1634,-73.824535,-147.436751,0,1,0


In [446]:
X_train.drop(['ts', 'imei'], inplace=True, axis=1)

In [447]:
def outlier_detection(df):
    for col in df.select_dtypes(include=[np.number]).columns:
        lower_bound = df[col].quantile(0.05)
        upper_bound = df[col].quantile(0.95)

        df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
        df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
    
    return df

def fill_missing_values(df):
    for col in df.select_dtypes(include=[np.number]).columns:
        df[col] = df[col].fillna(df[col].median())

    return df

In [448]:
X_train = fill_missing_values(X_train)
X_train = outlier_detection(X_train)

### (C)

In [449]:
standard_columns, minmax_columns, power_columns, quantile_columns = [], [], [], []

def select_transformer(column):
    skewness = column.skew()
    negative_values = (column < 0).any()
    
    iqr = column.quantile(0.75) - column.quantile(0.25)
    range = column.max() - column.min()
    large_range = range > iqr * 2
    
    if abs(skewness) >= 1.5 and not negative_values:
        power_columns.append(column.name)
        return
    
    if (column.max() > 1 or column.min() < 0) and (0.25 < abs(skewness) < 1.5):
        minmax_columns.append(column.name)
        return
    
    if large_range or (0.25 < abs(skewness) < 1.5):
        quantile_columns.append(column.name)
        return
    
    if abs(skewness) <= 0.25:
        standard_columns.append(column.name)
        return

for col in X_train.columns:
    select_transformer(X_train[col])

print('StandardScaler Columns:\t\t', standard_columns)
print('MinMaxScaler Columns:\t\t', minmax_columns)
print('PowerTransformer Columns:\t', power_columns)
print('QuantileTransformer Columns:\t', quantile_columns)

StandardScaler Columns:		 ['c.UCMobile.intl', 'c.updateassist', 'c.UCMobile.x86', 'p.android.defcontainer', 'p.google', 'p.android.gms', 'p.olauncher', 'p.android.vending', 'p.browser.provider', 'p.process.gapps', 'p.gms.persistent', 'p.simulator']
MinMaxScaler Columns:		 ['p.android.chrome', 'p.android.documentsui', 'p.android.gm']
PowerTransformer Columns:	 ['p.notifier']
QuantileTransformer Columns:	 ['c.android.gm', 'c.android.youtube', 'c.katana', 'c.dogalize', 'c.android.chrome', 'c.android.vending', 'c.raider', 'p.system', 'p.android.externalstorage', 'p.android.packageinstaller', 'p.android.settings', 'p.inputmethod.latin', 'p.katana', 'p.dogalize']


In [450]:
preprocessor = ColumnTransformer(
    transformers=[
        ('standard', StandardScaler(), standard_columns),
        ('minmax', MinMaxScaler(), minmax_columns),
        ('power', PowerTransformer(method='yeo-johnson'), power_columns),
        ('quantile', QuantileTransformer(output_distribution='normal'), quantile_columns)
    ],
    remainder='passthrough'
)

X_train_transformed = preprocessor.fit_transform(X_train)

### (D)

## 2. Výber atribútov pre strojové učenie

### (A)
#### 3.3.1 Informatívne atribúty k predikovanej premennej pre strojové učenie

##### Metóda pomocou random forest

In [451]:
rf = RandomForestRegressor()
rf.fit(X_train_transformed, y_train)

rf_features_sorted = sorted(
    list(zip([col for col in X_train.columns if col not in ['ts', 'imei']], rf.feature_importances_)),
    key=lambda x: x[1],
    reverse=True
)[:10]
rf_features = {name: importance for name, importance in rf_features_sorted}

print("Tabuľka atribútov podľa RandomForestRegressor:")
df_rf = pd.DataFrame(rf_features_sorted, columns=["Nazov atributu", "Random Forest hodnota"])
print(df_rf)
print("\n")

Tabuľka atribútov podľa RandomForestRegressor:
               Nazov atributu  Random Forest hodnota
0           p.android.vending               0.362649
1                    p.katana               0.163863
2       p.android.documentsui               0.047621
3         p.inputmethod.latin               0.046382
4  p.android.packageinstaller               0.039236
5                  p.notifier               0.034063
6   p.android.externalstorage               0.027874
7          p.android.settings               0.021351
8                 p.olauncher               0.019463
9          p.browser.provider               0.016513




##### Metóda pomocou Lasso Regression

In [452]:
lasso = Lasso(alpha=0.01) 
lasso.fit(X_train_transformed, y_train)

importance = pd.Series(lasso.coef_, index=[col for col in X_train.columns if col not in ['ts', 'imei']])

lasso_features = importance[importance != 0].nlargest(10)

print("10 najdôležitejších atribútov pomocou Lasso Regression:")
print(lasso_features)

10 najdôležitejších atribútov pomocou Lasso Regression:
p.android.packageinstaller    0.290033
p.katana                      0.063865
p.browser.provider            0.039718
p.android.externalstorage     0.020939
p.dogalize                    0.003143
p.android.gms                -0.009040
p.olauncher                  -0.015135
p.inputmethod.latin          -0.023858
p.notifier                   -0.042290
p.android.vending            -0.046421
dtype: float64


##### Metóda pomocou Select from model

In [ ]:
feature_names = X_train.columns

model = RandomForestClassifier()
model.fit(X_train_transformed, y_train)

selector = SelectFromModel(model, threshold=0, prefit=True)
X_selected = selector.transform(X_train_transformed)

selected_features_mask = selector.get_support()

selected_feature_names = feature_names[selected_features_mask]

selected_feature_importances = model.feature_importances_[selected_features_mask]

importance = pd.Series(selected_feature_importances, index=selected_feature_names)

select_from_model_features = importance.nlargest(10)

print("10 najdôležitejších atribútov pomocou Select from model: \n")
print(select_from_model_features)

10 najdôležitejších atribútov pomocou Select from model: 

p.android.vending             0.154287
p.katana                      0.146509
p.notifier                    0.116836
p.android.packageinstaller    0.068208
p.inputmethod.latin           0.062886
p.android.documentsui         0.059023
p.browser.provider            0.035289
p.android.externalstorage     0.033862
p.olauncher                   0.032160
p.android.settings            0.031330
dtype: float64


### (B)

In [454]:
all_features = set(rf_features.keys()) | set(lasso_features.keys()) | set(select_from_model_features.keys())

data = []
for feature in all_features:
    rf_value = rf_features.get(feature, "---")
    lasso_value = lasso_features.get(feature, "---")
    select_from_model_value = select_from_model_features.get(feature, "---")
    data.append((feature, rf_value, lasso_value, select_from_model_value))

df_combined = pd.DataFrame(data, columns=["Nazov atributu", "Random Forest hodnota", "Lasso hodnota", "Select from model hodnota"])
df_combined["Random Forest hodnota"] = pd.to_numeric(df_combined["Random Forest hodnota"], errors='coerce')
df_combined["Lasso hodnota"] = pd.to_numeric(df_combined["Lasso hodnota"], errors='coerce')

df_combined.sort_values(by="Random Forest hodnota", ascending=False, inplace=True)


df_combined.replace("x", 0, inplace=True)

df_combined["Select from model hodnota"] = pd.to_numeric(df_combined["Select from model hodnota"], errors='coerce')
df_combined["Select from model hodnota"] = df_combined["Select from model hodnota"]

df_combined["Výsledná hodnota"] = (
    pd.to_numeric(df_combined["Random Forest hodnota"], errors='coerce').fillna(0)
    + pd.to_numeric(df_combined["Lasso hodnota"], errors='coerce').fillna(0)
    + df_combined["Select from model hodnota"].fillna(0)
)

df_combined.sort_values(by="Výsledná hodnota", ascending=False, inplace=True)
df_combined.fillna(0, inplace=True)
print("Kombinovaná tabuľka atribútov podľa dôležitosti: \n")
print(df_combined.to_string(index=False))

Kombinovaná tabuľka atribútov podľa dôležitosti: 

            Nazov atributu  Random Forest hodnota  Lasso hodnota  Select from model hodnota  Výsledná hodnota
         p.android.vending               0.362649      -0.046421                   0.154287          0.470516
p.android.packageinstaller               0.039236       0.290033                   0.068208          0.397478
                  p.katana               0.163863       0.063865                   0.146509          0.374237
                p.notifier               0.034063      -0.042290                   0.116836          0.108609
     p.android.documentsui               0.047621       0.000000                   0.059023          0.106643
        p.browser.provider               0.016513       0.039718                   0.035289          0.091521
       p.inputmethod.latin               0.046382      -0.023858                   0.062886          0.085410
 p.android.externalstorage               0.027874       0.020939     

### (C)

## 3. Replikovateľnosť predspracovania

### (A)

### (B)

<small>Percentuálny podiel práce členov na projekte: Vlastislav Pokorný: 50%, Leon Radó: 50%</small>